'rm' is not recognized as an internal or external command,
operable program or batch file.


Note: you may need to restart the kernel to use updated packages.


'F:\anaconda\anaconda' is not recognized as an internal or external command,
operable program or batch file.


ModuleNotFoundError: No module named 'comet_ml'

In [3]:
# import the necessary packages
import tensorflow as tf
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import numpy as np
import glob
#import cv2
import os
from numpy import *


import datetime

def process_house_attributes(df, train, test):
	# initialize the column names of the continuous data
	continuous = ["t_web", "t_uf", "t_lf"]
	# performin min-max scaling each continuous feature column to
	# the range [0, 1]
	cs = MinMaxScaler()
	trainX = cs.fit_transform(train[continuous])
	testX = cs.transform(test[continuous])
	print(testX)
	print(trainX)   
	return (trainX, testX)



# import the necessary packages
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
def create_mlp(dim, regress=False):
	# define our MLP network
	model = Sequential()
	model.add(Dense(8, input_dim=dim, activation="relu"))
	model.add(Dense(4, activation="relu"))
	# check to see if the regression node should be added
	if regress:
		model.add(Dense(1, activation="linear"))
	# return our model
	return model



# import the necessary packages
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
#from pyimagesearch import datasets
#from pyimagesearch import models
import numpy as np
import argparse
import locale
import os

%load_ext tensorboard

# construct the argument parser and parse the arguments
##ap = argparse.ArgumentParser()
##ap.add_argument("-d", "--dataset", type=str, required=True,
##	help="path to input dataset of house images")
##args = vars(ap.parse_args())


# construct the path to the input .txt file that contains information
# on each house in the dataset and then load the dataset
print("[INFO] loading house attributes...")
##inputPath = os.path.sep.join([args["dataset"], "HousesInfo.txt"])
#df = load_house_attributes()
cols = ["t_web", "t_uf", "t_lf", "e_s"]
df = pd.read_csv(r"D:\MTP\my project\detail.csv", sep=",", header=None, names=cols)
# construct a training and testing split with 75% of the data used
# for training and the remaining 25% for evaluation
print("[INFO] constructing training/testing split...")
(train, test) = train_test_split(df, test_size=0.25, random_state=42)

# find the largest house price in the training set and use it to
# scale our house prices to the range [0, 1] (this will lead to
# better training and convergence)
EqStress = train["e_s"].max()
trainY = train["e_s"] / EqStress
testY = test["e_s"] / EqStress


# process the house attributes data by performing min-max scaling
# on continuous features, one-hot encoding on categorical features,
# and then finally concatenating them together
print("[INFO] processing data...")
(trainX, testX) = process_house_attributes(df, train, test)



# create our MLP and then compile the model using mean absolute
# percentage error as our loss, implying that we seek to minimize
# the absolute percentage difference between our price *predictions*
# and the *actual prices*
model = create_mlp(trainX.shape[1], regress=True)
opt = Adam(lr=1e-3, decay=1e-3 / 200)
model.compile(loss="mean_absolute_percentage_error", optimizer=opt)

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

# train the model


print("[INFO] training model...")
model.fit(x=trainX, y=trainY, 
	validation_data=(testX, testY),
	epochs=200, batch_size=8)



# make predictions on the testing data
print("[INFO] predicting Equivalent stress...")
preds = model.predict(testX)
#print(preds)

x_in =np.array([[0.5,0.5,0.5 ]])
x_in.reshape(3,)
y_res=model.predict(x_in)
y_res.flatten()
tY = y_res / EqStress
diff = y_res - tY
pDiff = (diff / tY) 
absPDiff = np.abs(pDiff)

print(absPDiff)
# compute the difference between the *predicted* house prices and the
# *actual* house prices, then compute the percentage difference and
# the absolute percentage difference
diff = preds.flatten() - testY
percentDiff = (diff / testY) * 100
absPercentDiff = np.abs(percentDiff)
# compute the mean and standard deviation of the absolute percentage
# difference
mean = np.mean(absPercentDiff)
std = np.std(absPercentDiff)
# finally, show some statistics on our model
locale.setlocale(locale.LC_ALL, "en_US.UTF-8")
print("[INFO] avg. Equivalent Stress: {}, std Equivalent Stress: {}".format(
	locale.currency(df["e_s"].mean(), grouping=True),
	locale.currency(df["e_s"].std(), grouping=True)))
print("[INFO] mean: {:.2f}%, std: {:.2f}%".format(mean, std))





The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
[INFO] loading house attributes...
[INFO] constructing training/testing split...
[INFO] processing data...
[[0.5  0.5  0.5 ]
 [0.75 0.25 0.75]
 [0.   1.   1.  ]
 [0.5  0.75 0.  ]
 [0.25 0.75 0.75]
 [0.   0.   0.75]
 [0.25 0.25 0.5 ]
 [0.   0.25 0.5 ]
 [1.   1.   0.75]
 [0.75 0.   0.75]]
[[0.75 0.5  0.  ]
 [0.25 0.75 1.  ]
 [0.75 0.25 1.  ]
 [0.5  0.5  1.  ]
 [0.75 1.   0.5 ]
 [0.25 1.   0.  ]
 [0.5  0.5  0.  ]
 [0.   0.   0.  ]
 [0.   0.   0.5 ]
 [0.25 0.   0.25]
 [0.5  0.25 0.75]
 [0.   0.25 0.25]
 [0.75 0.75 0.75]
 [1.   0.25 0.  ]
 [0.   0.5  0.5 ]
 [0.25 0.75 0.25]
 [0.75 0.25 0.25]
 [0.   0.75 0.75]
 [0.5  1.   0.5 ]
 [1.   0.75 0.5 ]
 [0.5  0.25 0.25]
 [0.5  0.   0.5 ]
 [1.   0.5  0.25]
 [0.75 0.75 0.25]
 [1.   0.   1.  ]
 [0.25 0.5  0.75]
 [0.5  1.   0.25]
 [0.25 0.25 0.75]
 [1.   0.5  0.5 ]]
[INFO] training model...
Train on 29 samples, validate on 10 samples
Epoch 1/200
29/29 [===========

In [8]:
%tensorboard --logdir logs/fit


Reusing TensorBoard on port 6006 (pid 10908), started 0:01:03 ago. (Use '!kill 10908' to kill it.)